# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import torch
print(torch.cuda.device_count())

1


In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.65it/s]


In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是八卦之一，代表阳、强、刚、动、进取、创造等含义。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雷等自然现象，以及君主、领导、父亲等权力和威严。乾卦的卦象是象征天空中的雷声，意味着力量和威严，也象征着积极向上、勇往直前的精神。在道家思想中，乾卦与坤卦相对，代表了宇宙中阴阳两极的作用和协调。在儒家思想中，乾卦象征着君子的品德，如忠诚、敬慎、刚毅等。在易经中，乾卦的卦辞：“天行健，君子以自强不息；地势坤，君子以厚德载物。”意味着君子应该效仿天的行健，不断自强，效仿地的坤德，厚植德行，以达到人生境界的提升。


In [9]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是八卦之一，由两个阴爻夹一个阳爻构成，象征着地、水、云等自然现象，以及军队、众、众志成城等含义。地水师卦象征着众志成城、团结合作的力量，同时也象征着柔顺、谦卑、低调的品质。在易经中，师卦的卦辞：“天行健，君子以自强不息；地势坤，君子以厚德载物。”意味着君子应该效仿天的行健，不断自强，效仿地的坤德，厚植德行，以达到人生境界的提升。在战争中，地水师卦象征着军队团结合作、众志成城的力量，能够带来胜利。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [10]:
from peft import PeftModel, PeftConfig

epochs = 50
timestamp = "20240129_073900"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [11]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [12]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易六十四卦中的第一卦，象征着天。由六个阳爻构成，具有刚强、键健的特性，预示着大吉大利，有利于事情的发展。此卦教导人们要自强不息，遵循天道的德行。在具体应用中，乾卦提醒人们要行正道，保持心境平和，避免骄傲自满，才能充分发挥才智，成就大的事业。同时，也要注意盛极必衰，提高警惕，小心谨慎。总的来说，乾卦是一种鼓励人们积极进取，不断自我提升的卦象。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易六十四卦中的第一卦，象征着天。由六个阳爻构成，具有刚强、键健的特性，预示着大吉大利，有利于事情的发展。此卦教导人们要自强不息，遵循天道的德行。在具体应用中，乾卦提醒人们要行正道，保持心境平和，避免骄傲自满，才能充分发挥才智，成就大的事业。同时，也要注意盛极必衰，提高警惕，小心谨慎。总的来说，乾卦是一种鼓励人们积极进取，不断自我提升的卦象。


In [13]:
base_response, ft_response = compare_chatglm_results(query="地水师卦")

问题：地水师卦

原始输出：
地水师卦是周易中的一个卦象，由下卦地（象征着大地）和上卦水（象征着流动的水）组成。这一卦象象征着时间和空间的流转，以及人类与自然的关系。在中医哲学中，地水师卦被认为是一种良好的卦象，象征着健康、繁荣和顺利。此卦大象象征着智慧和判断力，提醒人们要善于学习和成长。此外，地水还象征着生活和政治的变革，提醒人们要善于适应和处理复杂的情况。总的来说，地水师卦是一种积极的卦象，鼓励人们不断学习和进步，适应生活的变化。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦位 地水师卦位是一种卦象，由两个异卦相叠而成：下卦为坤，上卦为坎。这一卦的核心概念是'地'和'水'，象征着地球上的土壤和水分。它象征着顺从和遵循自然规律，因为地水一起流动，形成一种和谐统一的景象。这种卦象在哲学上提醒我们，我们应该尊重自然，而不是试图逆行。此外，地水师卦也提醒我们，在生活和工作中，我们应该适应环境，而不是强行改变它。总的来说，地水师卦是一种提醒我们尊重自然、顺应自然的卦象。它告诉我们，只有这样，我们才能真正实现个人的成长和事业的成功。


In [14]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")


问题：天水讼卦

原始输出：
天水讼卦是周易中的一个卦象，由乾卦和坎卦构成。这一卦象象征着争端和诉讼，提示人们在处理问题和决策时要慎重又慎重，避免纠纷和冲突。同时，它也提醒我们要保持谦逊和宽容，避免过分强调自身利益，以求和谐共处。《象辞》中提到'君子以做事谋始，'这意味着我们在处理事务时应该有长远的眼光和计划，而不是只看到眼前的利益。此外，此卦还告诫我们，虽然有时候可能会遇到困难和挫折，但只要我们坚持不懈，最终还是会有收获的。总的来说，天水讼卦给我们的启示是，在面对问题和挑战时，我们应该保持冷静和理智，避免争斗和冲突，以求达到最好的结果。


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦卦象解析 讼卦是周易中的一个异卦相叠的卦象，由乾卦（天）和坎卦（水）构成。这一卦象象征着争端和诉讼，提示人们在处理问题和决策时要慎重又慎重，避免纠纷和冲突。同时，它也提醒我们要保持谦逊和宽容，避免过分强调自身利益，以求和谐共处。《象辞》中提到'君子以做事谋始，'这意味着我们在处理事务时应该有长远的眼光和计划，而不是只看到眼前的利益。此外，此卦还告诫我们，虽然有时候可能会遇到困难和挫折，但只要我们坚持不懈，最终还是会有收获的。总的来说，讼卦给我们的启示是，在面对问题和挑战时，我们应该保持冷静和理智，避免争斗和冲突，以求达到最好的结果。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [15]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240129_054318"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [16]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [17]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易六十四卦中的第一卦，象征着天。由六个阳爻构成，具有刚强、键健的特性，预示着大吉大利，有利于事情的发展。此卦教导人们要自强不息，遵循天道的德行。在具体应用中，乾卦提醒人们要行正道，保持心境平和，避免骄傲自满，才能充分发挥才智，成就大的事业。同时，也要注意盛极必衰，提高警惕，小心谨慎。总的来说，乾卦是一种鼓励人们积极进取，不断自我提升的卦象。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易六十四卦中的第一卦，象征着天。由六个阳爻构成，具有刚强、键健的特性，预示着大吉大利，有利于事情的发展。此卦教导人们要自强不息，遵循天道的德行。在具体应用中，乾卦提醒人们要行正道，保持心境平和，避免骄傲自满，才能充分发挥才智，成就大的事业。同时，也要注意盛极必衰，提高警惕，小心谨慎。总的来说，乾卦是一种鼓励人们积极进取，不断自我提升的卦象。


In [18]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地震卦是周易中的一个异卦相叠的卦象，由乾卦（天）和坎卦（水）构成。这一卦象象征着地震，意味着天地之间的巨大力量。地震卦提醒人们在面对自然力量时，要保持警惕和谨慎。君子应该学习天地的精神，积极应对挑战，而不是过分 Trust in human intelligence。在决策时，要考虑长远和根本的利益，而不是短视和眼前的好处。地震卦 also告诫人们要珍惜生命，保护自然，实现和谐共处。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 师卦是周易中的一个重要卦象，由两个异卦相叠而成：下卦为坎（水），上卦为坤（地）。这一卦象象征着“师”，即军队，寓意着兵力和农力的结合。在战争策略上，坎卦象征着水和兵力的流动，而坤卦象征着地和兵力的聚集。师卦的核心哲学是：虽然兵力不同，但通过合理的结合和运用，可以使双方实力相当。在和平时期，它象征着智慧和知识，而在生活中，它则鼓励我们学会宽容和团结。此外，在军事策略中，坎卦的流动特性使得兵力可以迅速调动，而坤卦的稳定性则使得兵力可以得到有效指挥。因此，师卦是一种具有深刻哲学意义的卦象，既适用于战争，也适用于其他领域。


In [19]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一个异卦相叠的卦象，由乾卦（天）和坎卦（水）构成。这一卦象象征着争端和诉讼，提示人们在处理问题和决策时要慎重又慎重，避免纠纷和冲突。同时，它也提醒我们要保持谦逊和宽容，避免过分强调自身利益，以求和谐共处。《象辞》中提到'君子以做事谋始，'这意味着我们在处理事务时应该有长远的眼光和计划，而不是只看到眼前的利益。此外，此卦还告诫我们，虽然有时候可能会遇到困难和挫折，但只要我们坚持不懈，最终还是会有收获的。总的来说，讼卦给我们的启示是，在面对问题和挑战时，我们应该保持冷静和理智，避免争斗和冲突，以求达到最好的结果。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么? 讼卦是周易中的一个异卦相叠的卦象，由乾卦（天）和坎卦（水）构成。这一卦象象征着争端和诉讼，提示人们在处理问题和决策时要慎重又慎重，避免纠纷和冲突。同时，它也提醒我们要保持谦逊和宽容，避免过分强调自身利益，以求和谐共处。《象辞》中提到'君子以做事谋始，'这意味着我们在处理事务时应该有长远的眼光和计划，而不是只看到眼前的利益。此外，此卦还告诫我们，虽然有时候可能会遇到困难和挫折，但只要我们坚持不懈，最终还是会有收获的。总的来说，讼卦给我们的启示是，在面对问题和挑战时，我们应该保持冷静和理智，避免争斗和冲突，以求达到最好的结果。
